## Database: CITY VIEW

This tool set up different VIEWS (union of different indicators)

#----------------------------------------#

## DATABASE connection:

In [ ]:
from src.db_connect import create_engine

################################################## SET postgre-sql connection:

engine_postgresql = create_engine('path/to/database.ini')

print ("Engine started..")

#### (1.1)  VIEW I for the year 2018 

The different city indicators (values) will be LEFT joined to every city (729 cities) in the main urban-audit city list: 
[lut.l_city_urau2021]

The name of the new view will be cube.city_view_01_2018


In [ ]:
# (1.2)  VIEW !


# OUTPUT VIEW name:
schema = "public"
name_of_view = "city_2018_demo_view"


# (1) Removing old view - if Exists:


# removing table:...........................................S
query_drop = ("DROP VIEW IF EXISTS "+schema+"."+name_of_view+";")
connection = engine_postgresql.raw_connection()
cursor = connection.cursor()
cursor.execute(query_drop)
connection.commit()
# removing table:...........................................E


# (2) create new VIEWS
query = ("""      

  CREATE OR REPLACE VIEW """ + schema + "." + name_of_view + """
 AS

SELECT
    l_city_urau2021.city_code,
    l_city_urau2021.urau_code,
    l_city_urau2021.urau_catg,
    l_city_urau2021.cntr_code,
    l_city_urau2021.urau_name,
    l_city_urau2021.fua_code,
    l_city_urau2021.area_sqm,
    l_city_urau2021.nuts3_2021,
    l_city_urau2021.shape_area / 10000.00 as city_area_ha,
    l_city_urau2021._wgs84x,
    l_city_urau2021._wgs84y,
    l_city_urau2021._laeax,
    l_city_urau2021._laeay,
    --- DEM:
    c_city_dem.dem_mean,
    --- ENV Zone:
    lut.l_env_zones.ez_code,
    --- IMPERVIOUSNESS:
    100/l_city_urau2021.shape_area * 10000 * c_city_hrl_imperviousness_2018.parameter_value as imd_percent_2018,
    --- TREECOVER:
    100/l_city_urau2021.shape_area * 10000 * c_city_hrl_treecover_2018.parameter_value as treecover_percent_2018,
    --- Urban Atlas 2018
    class_11100,
    class_11210,
    class_11220,
    class_11230,
    class_11240,
    class_11300,
    class_12100,
    class_12210,
    class_12220,
    class_12230,
    class_12300,
    class_12400,
    class_13100,
    class_13300,
    class_13400,
    class_14100,
    class_14200,
    class_21000,
    class_22000,
    class_23000,
    class_24000,
    class_25000,
    class_31000,
    class_32000,
    class_33000,
    class_40000,
    class_50000,
    CASE WHEN class_50000> 0 THEN 
		100.00/(
		coalesce(class_11100,0)+
		coalesce(class_11210,0)+
		coalesce(class_11220,0)+
		coalesce(class_11230,0)+
		coalesce(class_11240,0)+
		coalesce(class_11300,0)+
		coalesce(class_12100,0)+
		coalesce(class_12210,0)+
		coalesce(class_12220,0)+
		coalesce(class_12230,0)+
		coalesce(class_12300,0)+
		coalesce(class_12400,0)+
		coalesce(class_13100,0)+
		coalesce(class_13300,0)+
		coalesce(class_13400,0)+
		coalesce(class_14100,0)+
		coalesce(class_14200,0)+
		coalesce(class_21000,0)+
		coalesce(class_22000,0)+
		coalesce(class_23000,0)+
		coalesce(class_24000,0)+
		coalesce(class_25000,0)+
		coalesce(class_31000,0)+
		coalesce(class_32000,0)+
		coalesce(class_33000,0)+
		coalesce(class_40000,0)+
		coalesce(class_50000,0)) * coalesce(class_50000,0)
 else 0 end as urban_blue_percent,
    CASE WHEN (coalesce(class_11230,0)+
			coalesce(class_11240,0)+
			coalesce(class_14100,0)+
			coalesce(class_14200,0)+
			coalesce(class_21000,0)+
			coalesce(class_22000,0)+
			coalesce(class_23000,0)+
			coalesce(class_24000,0)+
			coalesce(class_25000,0)+
			coalesce(class_31000,0)+
			coalesce(class_32000,0)+
			coalesce(class_33000,0)+
			coalesce(class_40000,0))> 0 THEN 
		100.00/
		(
		coalesce(class_11100,0)+
		coalesce(class_11210,0)+
		coalesce(class_11220,0)+
		coalesce(class_11230,0)+
		coalesce(class_11240,0)+
		coalesce(class_11300,0)+
		coalesce(class_12100,0)+
		coalesce(class_12210,0)+
		coalesce(class_12220,0)+
		coalesce(class_12230,0)+
		coalesce(class_12300,0)+
		coalesce(class_12400,0)+
		coalesce(class_13100,0)+
		coalesce(class_13300,0)+
		coalesce(class_13400,0)+
		coalesce(class_14100,0)+
		coalesce(class_14200,0)+
		coalesce(class_21000,0)+
		coalesce(class_22000,0)+
		coalesce(class_23000,0)+
		coalesce(class_24000,0)+
		coalesce(class_25000,0)+
		coalesce(class_31000,0)+
		coalesce(class_32000,0)+
		coalesce(class_33000,0)+
		coalesce(class_40000,0)+
		coalesce(class_50000,0)) * 
		  (coalesce(class_11230,0)+
			coalesce(class_11240,0)+
			coalesce(class_14100,0)+
			coalesce(class_14200,0)+
			coalesce(class_21000,0)+
			coalesce(class_22000,0)+
			coalesce(class_23000,0)+
			coalesce(class_24000,0)+
			coalesce(class_25000,0)+
			coalesce(class_31000,0)+
			coalesce(class_32000,0)+
			coalesce(class_33000,0)+
			coalesce(class_40000,0))
 else 0 end as urban_green_percent,
    --- 2m temp 2018
    temp_mean."2018" as avg_2m_temp_kelvin_2018,
    --Number of sommer days 2018
    c_city_era5_summer_days."2018" as number_of_summer_days_2018,
    --Number of tropical nights
    c_city_era5_tropical_nights.parameter_value as number_of_tropical_nights_2018,
    --Number of utci heat nights
    coalesce(utci_heat_nights_2018,0) as utci_heat_nights_2018,
    coalesce(c_coastal_cities.parameter_value,0) as coastal_city,
    -- urban audit:
    DE1001V_2018,
    DE1028V_2018,
    DE1055V_2018,
    EC1174V_2018,
    EC1010V_2018,
    EC1020I_2018,
    EC3040V_2018,
    SA2013V_2018,
    DE1028I_2018,
    DE1055I_2018
FROM lut.l_city_urau2021
    --DEM:
    left join cube.c_city_dem on c_city_dem.city_code   = l_city_urau2021.city_code
    --ENV ZONES:
    left join cube.c_city_env_zones on c_city_env_zones.city_code    = l_city_urau2021.city_code
    --ENV ZONES lookup table:    
    left join lut.l_env_zones on l_env_zones.ez_raster_code = c_city_env_zones.parameter_value
    --imperviousness_2018:
    left join cube.c_city_hrl_imperviousness_2018 on c_city_hrl_imperviousness_2018.city_code = l_city_urau2021.city_code
    --tree cover_2018:
    left join cube.c_city_hrl_treecover_2018 on c_city_hrl_treecover_2018.city_code = l_city_urau2021.city_code
    --Urban atlas 2018 area per class:
    left join
    (
		SELECT
        city_code,
        coalesce(max(CASE WHEN ua12_18_code = '11100' THEN (parameter_value) END),0) as class_11100, --- - Continuous Urban Fabric (S.L. > 80%)',
        coalesce(max(CASE WHEN ua12_18_code = '11210' THEN (parameter_value) END),0) as class_11210, --- - Discontinuous Dense Urban Fabric (S.L. : 50% - 80%)',
        coalesce(max(CASE WHEN ua12_18_code = '11220' THEN (parameter_value) END),0) as class_11220, --- - Discontinuous Medium Density Urban Fabric (S.L. : 30% - 50%)',
        coalesce(max(CASE WHEN ua12_18_code = '11230' THEN (parameter_value) END),0) as class_11230, --- - Discontinuous Low Density Urban Fabric (S.L. : 10% - 30%)',
        coalesce(max(CASE WHEN ua12_18_code = '11240' THEN (parameter_value) END),0) as class_11240, --- - Discontinuous Very Low Density Urban Fabric (S.L. < 10%)',
        coalesce(max(CASE WHEN ua12_18_code = '11300' THEN (parameter_value) END),0) as class_11300, --- - Isolated Structures',
        coalesce(max(CASE WHEN ua12_18_code = '12100' THEN (parameter_value) END),0) as class_12100, --- - Industrial, commercial, public, military and private units',
        coalesce(max(CASE WHEN ua12_18_code = '12210' THEN (parameter_value) END),0) as class_12210, --- - Fast transit roads and associated land',
        coalesce(max(CASE WHEN ua12_18_code = '12220' THEN (parameter_value) END),0) as class_12220, --- - Other roads and associated land',
        coalesce(max(CASE WHEN ua12_18_code = '12230' THEN (parameter_value) END),0) as class_12230, --- - Railways and associated land',
        coalesce(max(CASE WHEN ua12_18_code = '12300' THEN (parameter_value) END),0) as class_12300, --- - Port areas',
        coalesce(max(CASE WHEN ua12_18_code = '12400' THEN (parameter_value) END),0) as class_12400, --- - Airports',
        coalesce(max(CASE WHEN ua12_18_code = '13100' THEN (parameter_value) END),0) as class_13100, --- - Mineral extraction and dump sites',
        coalesce(max(CASE WHEN ua12_18_code = '13300' THEN (parameter_value) END),0) as class_13300, --- - Construction sites',
        coalesce(max(CASE WHEN ua12_18_code = '13400' THEN (parameter_value) END),0) as class_13400, --- - Land without current use',
        coalesce(max(CASE WHEN ua12_18_code = '14100' THEN (parameter_value) END),0) as class_14100, --- - Green urban areas',
        coalesce(max(CASE WHEN ua12_18_code = '14200' THEN (parameter_value) END),0) as class_14200, --- - Sports and leisure facilities',
        coalesce(max(CASE WHEN ua12_18_code = '21000' THEN (parameter_value) END),0) as class_21000, --- - Arable land (annual crops)',
        coalesce(max(CASE WHEN ua12_18_code = '22000' THEN (parameter_value) END),0) as class_22000, --- - Permanent crops (vineyards, fruit trees, olive groves)',
        coalesce(max(CASE WHEN ua12_18_code = '23000' THEN (parameter_value) END),0) as class_23000, --- - Pastures',
        coalesce(max(CASE WHEN ua12_18_code = '24000' THEN (parameter_value) END),0) as class_24000, --- - Complex and mixed cultivation patterns',
        coalesce(max(CASE WHEN ua12_18_code = '25000' THEN (parameter_value) END),0) as class_25000, --- - Orchards',
        coalesce(max(CASE WHEN ua12_18_code = '31000' THEN (parameter_value) END),0) as class_31000, --- - Forests',
        coalesce(max(CASE WHEN ua12_18_code = '32000' THEN (parameter_value) END),0) as class_32000, --- - Herbaceous vegetation associations (natural grassland, moors…)',
        coalesce(max(CASE WHEN ua12_18_code = '33000' THEN (parameter_value) END),0) as class_33000, --- - Open spaces with little or no vegetations',
        coalesce(max(CASE WHEN ua12_18_code = '40000' THEN (parameter_value) END),0) as class_40000, --- - Wetland',
        coalesce(max(CASE WHEN ua12_18_code = '50000' THEN (parameter_value) END),0) as class_50000
    --- - Water bodies'
    FROM cube.c_city_urban_atlas_2018
    group by city_code) ua18 on ua18.city_code = l_city_urau2021.city_code
    --2m mean temp 2018 :
    left join (
        SELECT city_code, "2018"
    FROM cube.c_city_era5_2m_temperature
    where parameter_id = 'city_era5_2m_temperature_yearly_mean'
	) temp_mean
    on left(temp_mean.city_code,6) = l_city_urau2021.city_code
    --Number of sommer days 2018 :
    left join cube.c_city_era5_summer_days on left(c_city_era5_summer_days.city_code,6) = l_city_urau2021.city_code
    --Number of tropical nights :
    left join cube.c_city_era5_tropical_nights on left(c_city_era5_tropical_nights.city_code,6)   = l_city_urau2021.city_code
    --Number of heat nights (UTCI):
    left join (   
			SELECT city_code, SUM("2018") as "utci_heat_nights_2018"
    FROM cube.c_city_utci_indicator
    where parameter_id	
				in 
					( 'utci_class_very_strong_heat_stress',
					'utci_class_strong_heat_stress',
					'utci_class_extreme_heat_stress')
    group by city_code
	) utci_heat_nights
    on left(utci_heat_nights.city_code,6) = l_city_urau2021.city_code
    left join cube.c_coastal_cities on left(cube.c_coastal_cities.city_code,6) = l_city_urau2021.city_code
    left join ( 
	SELECT city_code
, MAX (CASE when estat_code = 'DE1001V' THEN COALESCE("2018", "2017", "2019", "2020","2016", "2021","2015","2014","2013","2012") END) as DE1001V_2018
, MAX (CASE when estat_code = 'DE1028V' THEN COALESCE("2018", "2017", "2019", "2020","2016", "2021","2015","2014","2013","2012") END) as DE1028V_2018
, MAX (CASE when estat_code = 'DE1055V' THEN COALESCE("2018", "2017", "2019", "2020","2016", "2021","2015","2014","2013","2012") END) as DE1055V_2018
, MAX (CASE when estat_code = 'EC1174V' THEN COALESCE("2018", "2017", "2019", "2020","2016", "2021","2015","2014","2013","2012") END) as EC1174V_2018
, MAX (CASE when estat_code = 'EC1010V' THEN COALESCE("2018", "2017", "2019", "2020","2016", "2021","2015","2014","2013","2012") END) as EC1010V_2018
, MAX (CASE when estat_code = 'EC1020I' THEN COALESCE("2018", "2017", "2019", "2020","2016", "2021","2015","2014","2013","2012") END) as EC1020I_2018
, MAX (CASE when estat_code = 'EC3040V' THEN COALESCE("2018", "2017", "2019", "2020","2016", "2021","2015","2014","2013","2012") END) as EC3040V_2018
, MAX (CASE when estat_code = 'SA2013V' THEN COALESCE("2018", "2017", "2019", "2020","2016", "2021","2015","2014","2013","2012") END) as SA2013V_2018
, MAX (CASE when estat_code = 'DE1028I' THEN COALESCE("2018", "2017", "2019", "2020","2016", "2021","2015","2014","2013","2012") END) as DE1028I_2018
, MAX (CASE when estat_code = 'DE1055I' THEN COALESCE("2018", "2017", "2019", "2020","2016", "2021","2015","2014","2013","2012") END) as DE1055I_2018
    FROM cube.c_city_eurostat
    group by city_code
    order by city_code
	) urban_audit on left(urban_audit.city_code,6) = l_city_urau2021.city_code
         """)
# print (query)
connection = engine_postgresql.raw_connection()
cursor = connection.cursor()
cursor.execute(query)
connection.commit()

print("VIEW 1 produced!! ")

VIEW 1 produced!! 
